In [269]:
import requests
import json
import re
import base64

In [270]:
email = 'woojinp1994@gmail.com'

url = f'https://ciphersprint.pulley.com/{email}'
response = requests.get(url)
task = response.json()
print(task)

{'challenger': 'woojinp1994@gmail.com', 'encrypted_path': 'task_3e2ee3f39854232c21858528827031a0', 'encryption_method': 'nothing', 'expires_in': '60s', 'hint': 'go to /task_3e2ee3f39854232c21858528827031a0 to get the next challenge', 'instructions': 'encrypted_path is the path to your next challenge. encryption_method is how it was encrypted.', 'level': 0}


In [271]:
task_path = task['encrypted_path']

url = f'https://ciphersprint.pulley.com/{task_path}'
response = requests.get(url)
task = response.json()
print(task)

{'challenger': 'woojinp1994@gmail.com', 'encrypted_path': 'task_[53,102,54,55,50,49,56,49,55,48,48,51,48,50,53,52,55,48,56,53,56,56,98,54,52,52,54,101,101,97,53,102]', 'encryption_method': 'converted to a JSON array of ASCII values', 'expires_in': '15s', 'hint': 'you should write a program to fetch these endpoints and handle each encryption method you find.', 'instructions': 'encrypted_path is the path to your next challenge. encryption_method is how it was encrypted.', 'level': 1}


In [272]:
def decrypter_ascii(encrypted_path):
    # Get ASCII values from the encrypted path
    ascii_values = encrypted_path.split('[')[1].split(']')[0].split(',')
    # Convert ASCII values to characters and join them to form the decrypted path
    decrypted_path = ''.join(chr(int(x)) for x in ascii_values)
    return decrypted_path

task_path = task['encrypted_path']

task_path_decrypted = 'task_' + decrypter_ascii(task_path)
url = f'https://ciphersprint.pulley.com/{task_path_decrypted}'
response = requests.get(url)
task = response.json()
print(task)

{'challenger': 'woojinp1994@gmail.com', 'encrypted_path': 'task_40a9992ahc0f315f7g1638j25ff4cfikbal1f', 'encryption_method': 'inserted some non-hex characters', 'expires_in': '1s', 'hint': 'you should write a program to fetch these endpoints and handle each encryption method you find.', 'instructions': 'encrypted_path is the path to your next challenge. encryption_method is how it was encrypted.', 'level': 2}


In [273]:
def decrypter_nonhex(encrypted_string):
    # Fix link with regular expressions
    decrypted_path = re.sub(r'[^a-fA-F0-9]', '', encrypted_string)
    return decrypted_path

task_path = task['encrypted_path']
encrypted_string = task_path.split('_')[1]

task_path_decrypted = 'task_' + decrypter_nonhex(encrypted_string)
url = f'https://ciphersprint.pulley.com/{task_path_decrypted}'
response = requests.get(url)
task = response.json()
print(task)

{'challenger': 'woojinp1994@gmail.com', 'encrypted_path': 'task_73e8c25229836b21b7e95e68593c49f5', 'encryption_method': 'circularly rotated left by 17', 'expires_in': '1s', 'hint': 'you should write a program to fetch these endpoints and handle each encryption method you find.', 'instructions': "we've been alerted that you reached level 3, keep going!", 'level': 3}


In [274]:
def decrypter_rotated(encrypted_string, shift_amount):
    shift = int(shift_amount)  # Circular rotation 
    # we should shift string to the right, since the encryption shifted to the left
    decrypted_string = encrypted_string[-shift:] + encrypted_string[:-shift]
    return decrypted_string

task_path = task['encrypted_path']
encrypted_string = task_path.split('_')[1]
shift_amount = task['encryption_method'].split(' ')[-1]

task_path_decrypted = 'task_' + decrypter_rotated(encrypted_string, shift_amount)
url = f'https://ciphersprint.pulley.com/{task_path_decrypted}'
response = requests.get(url)
task = response.json()
print(task)

{'challenger': 'woojinp1994@gmail.com', 'encrypted_path': 'task_0ed5a571671a3a3f0c60e3d46a5ac3f5', 'encryption_method': 'hex decoded, encrypted with XOR, hex encoded again. key: secret', 'expires_in': '1s', 'hint': 'you should write a program to fetch these endpoints and handle each encryption method you find.', 'instructions': "we've been alerted that you reached level 4, keep going!", 'level': 4}


In [275]:
def decrypt_xor(encrypted_string, key):
    # Decode the hex string
    decoded_bytes = bytes.fromhex(encrypted_string)
    
    # Decrypt using XOR
    decrypted_bytes = bytes(c ^ key[i % len(key)] for i, c in enumerate(decoded_bytes))
    
    # Encode in hex again
    encoded_hex_string = decrypted_bytes.hex()
    
    return encoded_hex_string

# denote that the string should be bytes
key = b'secret'

task_path = task['encrypted_path']
encrypted_string = task_path.split('_')[1]

task_path_decrypted = 'task_' + decrypt_xor(encrypted_string, key)
url = f'https://ciphersprint.pulley.com/{task_path_decrypted}'
response = requests.get(url)
task = response.json()
print(task)

{'challenger': 'woojinp1994@gmail.com', 'encrypted_path': 'task_b72c47dc1be2f0045e43d367f0f59667', 'encryption_method': 'scrambled! original positions as base64 encoded messagepack: 3AAgCgsTEgIcAQkNFQgUBwUbDhAZFgAGDAQYGhceHwMPHRE=', 'expires_in': '1s', 'hint': 'you should write a program to fetch these endpoints and handle each encryption method you find.', 'instructions': "we've been alerted that you reached level 5, keep going!", 'level': 5}


In [276]:
def decrypt_scrambled(encrypted_string, messagepack):
    # Decode the base64 encoded message
    decoded_message = base64.b64decode(messagepack)
    
    # Interpret the decoded message as a list of integers representing original positions
    original_positions = list(decoded_message)[3:] # Remove first 3 from set because they're weird numbers. Not exactly sure what's going on there
    
    # Create dictionary for positions to encrypted string
    char_position_map = dict(zip(original_positions, encrypted_string))
    
    # Rearrange the characters following the position map
    decrypted_string = ''.join(char_position_map[pos] for pos in sorted(original_positions))
    
    return decrypted_string

task_path = task['encrypted_path']
encrypted_string = task_path.split('_')[1]
messagepack = task['encryption_method'].split(' ')[-1]

task_path_decrypted = 'task_' + decrypt_scrambled(encrypted_string, messagepack)
url = f'https://ciphersprint.pulley.com/{task_path_decrypted}'
response = requests.get(url)
task = response.json()
print(task)

{'challenger': 'woojinp1994@gmail.com', 'encrypted_path': 'task_f530b4ea053099e587817b55eb96ad01e6c4280ccc36bab9a5802614f2237b62', 'encryption_method': 'hashed with sha256, good luck', 'expires_in': '10000000s', 'hint': 'this is a gimmick, there is no way to solve this task.', 'instructions': "we've been alerted that you reached level 6, congratulations!", 'level': 6}
